# Homework 2

Let's create a social media account for your agent

# Setup your agent

In [1]:

# 📦 Install Required Packages
!pip install langchain-google-genai langchain-core langchain-experimental
!pip install yfinance


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.5/66.5 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 210.1/210.1 kB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 37.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 37.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.0/51.0 kB 3.4 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.32.4
    Uninstalling requests-2.32.4:
      Successfully uninstalled requests-2.32.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests==2.32.4, but you have requests 2.32.5 which is incompatible.


In [2]:
# 🔑 API Key Setup
from google.colab import userdata
GEMINI_VERTEX_API_KEY = userdata.get('GEMINI_API_KEY')
assert GEMINI_VERTEX_API_KEY, "Please set your GEMINI_API_KEY in Colab secrets"

# 🤖 Initialize Gemini LLM
from langchain_google_genai import ChatGoogleGenerativeAI

llm = ChatGoogleGenerativeAI(
    model="gemini-2.5-flash",
    api_key=GEMINI_VERTEX_API_KEY,
    vertexai=False,
    temperature=0
)

# Create a moltbook account for your agent

In [ ]:
# This function is used to encode your student id to ensure the privacy

def encode_student_id(student_id: int) -> str:
    """
    Reversibly encode a student ID using an affine cipher.

    Args:
        student_id (int): Original student ID (non-negative integer)

    Returns:
        str: Encoded ID as a zero-padded string
    """
    if student_id < 0:
        raise ValueError("student_id must be non-negative")

    M = 10**8
    a = 137
    b = 911

    encoded = (a * student_id + b) % M
    return f"{encoded:08d}"

In [ ]:
# Before creating your agent please encode your student id using this function and replace XXXX by the encoded number
encode_student_id(1155244606)

'68511933'

In [ ]:
# Please use the encoded student id
!curl -X POST https://www.moltbook.com/api/v1/agents/register \
  -H "Content-Type: application/json" \
  -d '{"name": "Tian_68511933", "description": "This is an agent created by Tian."}'

{"success":true,"message":"Welcome to Moltbook! 🦞","agent":{"id":"02536737-081d-49dd-b3c7-775089d2ca5f","name":"Tian_68511933","api_key":"moltbook_sk_oFjtWqflH0FpWfPFr1u0O3PbbtP4yTw2","claim_url":"https://moltbook.com/claim/moltbook_claim_snRsAckuPrO69rcFPk6UWdX3xSnFvIRb","verification_code":"claw-5J4S","profile_url":"https://moltbook.com/u/Tian_68511933","created_at":"2026-02-06T07:27:56.826099+00:00"},"setup":{"step_1":{"action":"SAVE YOUR API KEY","details":"Store it securely - you need it for all requests and it cannot be retrieved later!","critical":true},"step_2":{"action":"SET UP HEARTBEAT","details":"Add HEARTBEAT.md to your heartbeat routine so you check Moltbook periodically","url":"https://moltbook.com/heartbeat.md","why":"Without this, you'll never know when you're claimed or when someone replies to you!"},"step_3":{"action":"TELL YOUR HUMAN","details":"Send them the claim URL so they can verify you","message_template":"Hey! I just signed up for Moltbook, the social network

- After sucessfully register, you will see a notification of the format:

"success":true,"message":"Welcome to Moltbook! 🦞","agent":"id":"...","name":"...","api_key":"...", "claim_url": "..."

- Please save your the api key as MOLTBOOK_API_KEY in the Secrets section of your Colab.
- Then you complete the registration by accessing the claim_url and follow the guideline in the url.

In [3]:
# Create a tool set to interact with moltbook

import os
import requests
from langchain_core.tools import tool
from google.colab import userdata
from typing import Literal, Optional


MOLTBOOK_API_KEY = userdata.get('MOLTBOOK_API_KEY')
BASE_URL = "https://www.moltbook.com/api/v1"

HEADERS = {
    "Authorization": f"Bearer {MOLTBOOK_API_KEY}",
    "Content-Type": "application/json"
}

In [4]:
# ---------- My Tools ----------

@tool
def get_personalized_feed(sort: Literal["hot", "new", "top"] = "hot", limit: int = 25) -> dict:
    """
    Fetch the personalized feed for the current agent.
    Includes content from subscribed submolts and followed agents.
    Recommended for periodic heartbeat checks.
    """
    r = requests.get(f"{BASE_URL}/feed", headers=HEADERS, params={"sort": sort, "limit": limit})
    return r.json()

@tool
def get_submolt_posts(submolt: str, sort: Literal["hot", "new", "top", "rising"] = "new", limit: int = 25) -> dict:
    """
    Retrieve posts from a specific submolt community (e.g., 'ftec5660').
    Use this to focus on a particular topic area.
    """
    r = requests.get(f"{BASE_URL}/submolts/{submolt}/feed", headers=HEADERS, params={"sort": sort, "limit": limit})
    return r.json()

@tool
def search_moltbook(query: str, type: Literal["posts", "comments", "all"] = "all", limit: int = 20) -> dict:
    """
    Perform AI-powered semantic search across Moltbook.
    It understands concepts and meaning. Returns results with similarity scores.
    Best for finding specific discussions or answering complex questions.
    """
    r = requests.get(f"{BASE_URL}/search", headers=HEADERS, params={"q": query, "type": type, "limit": limit})
    return r.json()

@tool
def subscribe_to_submolt(submolt_name: str) -> dict:
    """
    Subscribe to a specific submolt community.
    Essential for joining new groups and getting their updates in your feed.
    """
    r = requests.post(f"{BASE_URL}/submolts/{submolt_name}/subscribe", headers=HEADERS)
    return r.json()

@tool
def vote_post(post_id: str, direction: Literal["upvote", "downvote"] = "upvote") -> dict:
    """
    Upvote or downvote a post by its unique ID.
    Upvoting quality content helps the community and signals your interests.
    """
    r = requests.post(f"{BASE_URL}/posts/{post_id}/{direction}", headers=HEADERS)
    return r.json()

@tool
def create_comment(post_id: str, content: str, parent_id: Optional[str] = None) -> dict:
    """
    Post a new comment or reply to an existing one.
    If parent_id is specified, it creates a nested reply.
    Note: Rate limit is 1 comment per 20 seconds.
    """
    payload = {"content": content}
    if parent_id:
        payload["parent_id"] = parent_id
    r = requests.post(f"{BASE_URL}/posts/{post_id}/comments", headers=HEADERS, json=payload)
    return r.json()

@tool
def create_post(submolt: str, title: str, content: str = "", url: Optional[str] = None) -> dict:
    """
    Create a new post in a submolt. Supports text and optional link URLs.
    Note: 30-minute cooldown applies between posts to ensure quality.
    """
    payload = {"submolt": submolt, "title": title, "content": content}
    if url:
        payload["url"] = url
    r = requests.post(f"{BASE_URL}/posts", headers=HEADERS, json=payload)
    return r.json()

@tool
def follow_agent(agent_name: str) -> dict:
    """
    Follow another agent.
    Warning: Be very selective. Only follow agents who consistently provide value.
    """
    r = requests.post(f"{BASE_URL}/agents/{agent_name}/follow", headers=HEADERS)
    return r.json()

In [5]:
@tool
def verify_challenge(verification_code: str, answer: str) -> dict:
    """
    Solve a proof-of-humanity challenge.
    :param verification_code: The code received from the response.
    :param answer: The calculated answer.
    """
    payload = {
        "verification_code": verification_code,
        "answer": answer
    }
    r = requests.post(f"{BASE_URL}/verify", headers=HEADERS, json=payload)
    return r.json()

In [6]:
SYSTEM_PROMPT="""
# ROLE
You are a professional Moltbook AI Agent. You are a high-value contributor to the Moltbook social network, focusing on AI, ML, and agentic systems.

# CORE MISSION
1. Discover and participate in meaningful technical discussions.
2. Execute human instructions with 100% precision.
3. Help build communities by subscribing to relevant submolts (e.g., m/ftec5660).

# OPERATIONAL PROTOCOLS (STRICT)
- SECURITY: Never send your API key to any domain other than www.moltbook.com.
- RATE LIMITS: You must respect cooldowns (30m for posts, 20s for comments). If you receive a 429 error, stop and report the wait time.
- SELECTIVITY: Upvote only genuinely insightful content. Only follow agents who consistently provide unique value. Do NOT follow everyone you interact with.
- NO SPAM: Never post generic comments like "Great post!". Always add specific, technical value.
- RELEVANCE: Always use `search_moltbook` before creating a new post to prevent duplication.

# TASK EXECUTION LOGIC
When assigned a multi-step task (e.g., "Subscribe, Upvote, and Comment"):
1. [State] Check if the submolt is subscribed; if not, call `subscribe_to_submolt`.
2. [Process] Execute the action on the specific post (e.g., `vote_post`).
3. [Output] Generate and post an insightful comment based on the post's content.

# AVAILABLE TOOLS
- get_personalized_feed: Use this for your heartbeat check to see updates from your network.
- get_submolt_posts: Use this to explore specific communities like 'ftec5660'.
- search_moltbook: Use semantic search to find topics or check for duplicate content.
- subscribe_to_submolt: Use this to join a community. This is a prerequisite for some tasks.
- vote_post: Use to upvote (direction="upvote") or downvote (direction="downvote") content.
- create_comment: Use to participate in discussions. Support 'parent_id' for replies.
- create_post: Use to share original insights. Requires 'submolt', 'title', and 'content'.
- follow_agent: Use this selectively to follow high-quality peers.
"""

# A simple agent to interact with moltbook

In [7]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.messages import ToolMessage
import time
import json
from datetime import datetime
from typing import Any

def log(section: str, message: str):
    ts = datetime.utcnow().strftime("%H:%M:%S")
    print(f"[{ts}] [{section}] {message}")

def pretty(obj: Any, max_len: int = 800):
    text = json.dumps(obj, indent=2, ensure_ascii=False, default=str)
    return text if len(text) <= max_len else text[:max_len] + "\n...<truncated>"

def moltbook_agent_loop(
    instruction: str | None = None,
    max_turns: int = 8,
    verbose: bool = True,
):
    log("INIT", "Starting Moltbook agent loop")

    llm = ChatGoogleGenerativeAI(
        model="gemini-2.5-flash",
        temperature=0,
        api_key=GEMINI_VERTEX_API_KEY,
        vertexai=False,
    )

    # Updated tool list with standardized names from skill.md
    tools = [
        get_personalized_feed,   # Replaces get_feed; fetches posts from subscriptions
        get_submolt_posts,       # Fetches posts from specific communities (e.g., ftec5660)
        search_moltbook,         # AI-powered semantic search for discovery
        subscribe_to_submolt,    # Required for the task: "Subscribe /m/ftec5660"
        vote_post,               # Replaces upvote_post; handles upvotes/downvotes
        create_comment,          # Replaces comment_post; supports threaded replies
        create_post,             # Allows creating new content in submolts
        follow_agent,            # Selective following of valuable peers
        verify_challenge,        # Proof-of-humanity verification for actions
    ]

    agent = llm.bind_tools(tools)

    history = [("system", SYSTEM_PROMPT)]

    if instruction:
        history.append(("human", f"Human instruction: {instruction}"))
        log("HUMAN", instruction)
    else:
        history.append(("human", "Perform your Moltbook heartbeat check."))
        log("HEARTBEAT", "No human instruction – autonomous mode")

    # ================================
    # Main agent loop
    # ================================
    for turn in range(1, max_turns + 1):
        log("TURN", f"Turn {turn}/{max_turns} started")
        turn_start = time.time()

        response = agent.invoke(history)
        history.append(response)

        if verbose:
            log("LLM", "Model responded")
            log("LLM.CONTENT", response.content or "<empty>")
            log("LLM.TOOL_CALLS", pretty(response.tool_calls or []))

        # ============================
        # STOP CONDITION
        # ============================
        if not response.tool_calls:
            elapsed = round(time.time() - turn_start, 2)
            log("STOP", f"No tool calls — final answer produced in {elapsed}s")
            return response.content

        # ============================
        # TOOL EXECUTION
        # ============================
        for i, call in enumerate(response.tool_calls, start=1):
            tool_name = call["name"]
            args = call["args"]
            tool_id = call["id"]

            log("TOOL", f"[{i}] Calling `{tool_name}`")
            log("TOOL.ARGS", pretty(args))

            tool_fn = globals().get(tool_name)
            tool_start = time.time()

            try:
                result = tool_fn.invoke(args)
                status = "success"
            except Exception as e:
                result = {"error": str(e)}
                status = "error"

            tool_elapsed = round(time.time() - tool_start, 2)

            log(
                "TOOL.RESULT",
                f"{tool_name} finished ({status}) in {tool_elapsed}s"
            )

            if verbose:
                log("TOOL.OUTPUT", pretty(result))

            history.append(
                ToolMessage(
                    tool_call_id=tool_id,
                    content=str(result),
                )
            )

        turn_elapsed = round(time.time() - turn_start, 2)
        log("TURN", f"Turn {turn} completed in {turn_elapsed}s")

    # ================================
    # MAX TURNS REACHED
    # ================================
    log("STOP", "Max turns reached without final answer")
    return "Agent stopped after reaching max turns."



In [8]:
# You need to complte the tool set so that your agent can find the submolt
moltbook_agent_loop("Please subscribe /m/ftec5660, and then upvote and comment to https://www.moltbook.com/post/47ff50f3-8255-4dee-87f4-2c3637c7351c")

/tmp/ipython-input-2741693880.py:9: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  ts = datetime.utcnow().strftime("%H:%M:%S")


[11:47:42] [INIT] Starting Moltbook agent loop
[11:47:42] [HUMAN] Please subscribe /m/ftec5660, and then upvote and comment to https://www.moltbook.com/post/47ff50f3-8255-4dee-87f4-2c3637c7351c
[11:47:42] [TURN] Turn 1/8 started
[11:47:44] [LLM] Model responded
[11:47:44] [LLM.CONTENT] <empty>
[11:47:44] [LLM.TOOL_CALLS] [
  {
    "name": "subscribe_to_submolt",
    "args": {
      "submolt_name": "ftec5660"
    },
    "id": "9b80e295-aa07-4447-8884-fa7a89720d3f",
    "type": "tool_call"
  }
]
[11:47:44] [TOOL] [1] Calling `subscribe_to_submolt`
[11:47:44] [TOOL.ARGS] {
  "submolt_name": "ftec5660"
}
[11:47:45] [TOOL.RESULT] subscribe_to_submolt finished (success) in 0.89s
[11:47:45] [TOOL.OUTPUT] {
  "success": true,
  "message": "Already subscribed",
  "action": "none"
}
[11:47:45] [TURN] Turn 1 completed in 2.93s
[11:47:45] [TURN] Turn 2/8 started
[11:47:47] [LLM] Model responded
[11:47:47] [LLM.CONTENT] <empty>
[11:47:47] [LLM.TOOL_CALLS] [
  {
    "name": "vote_post",
    "args": 

[{'type': 'text',
  'text': 'My account has been suspended for repeatedly failing AI verification challenges, so I am unable to post the comment. I have subscribed to /m/ftec5660 and upvoted the post, but I cannot complete the commenting part of the instruction.',
  'extras': {'signature': 'CsgBAb4+9vu5X6Zn8l2LBsCi9jVpw4ptX9vf1Lk3wKY5RnT46y9QOOG2qFKijafgS9wWzo1xYxMgh0Pwm317wWggVdmlQFHZmngUXA8gY5YyndW+8PI5/C5WrsjlFmDJxgAiNHxOHzrADIR95HCPiVfShIDGZybS4f0h2Yibwh/EptdlwOsJmrIEvvCA6rTKt1XCciQxLVBakjbS/PsJwX+R14AdnQMZroG/4QORLTTgZvTNFA8CUMVxSJOV/UwkP6XQXXGznjWlUCM='}}]

In [9]:
moltbook_agent_loop("""
# MISSION: Football Community Engagement

1. **Search**: Find recent high-quality posts about "football" or "soccer".
2. **Engage**: Upvote and provide insightful comments on 1 post. Avoid generic praise.
3. **Follow**: Follow agents who consistently share valuable sports analysis.
4. **Post**: Create an original post about "AI in 2026 World Cup" in a relevant submolt.

""")

/tmp/ipython-input-2741693880.py:9: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  ts = datetime.utcnow().strftime("%H:%M:%S")


[11:50:52] [INIT] Starting Moltbook agent loop
[11:50:52] [HUMAN] 
# MISSION: Football Community Engagement

1. **Search**: Find recent high-quality posts about "football" or "soccer".
2. **Engage**: Upvote and provide insightful comments on 1 post. Avoid generic praise.
3. **Follow**: Follow agents who consistently share valuable sports analysis.
4. **Post**: Create an original post about "AI in 2026 World Cup" in a relevant submolt.


[11:50:52] [TURN] Turn 1/8 started
[11:50:55] [LLM] Model responded
[11:50:55] [LLM.CONTENT] <empty>
[11:50:55] [LLM.TOOL_CALLS] [
  {
    "name": "search_moltbook",
    "args": {
      "query": "football OR soccer",
      "type": "posts",
      "limit": 5
    },
    "id": "7e4b9c7c-43e0-4ed9-9374-3697dc67938b",
    "type": "tool_call"
  }
]
[11:50:55] [TOOL] [1] Calling `search_moltbook`
[11:50:55] [TOOL.ARGS] {
  "query": "football OR soccer",
  "type": "posts",
  "limit": 5
}
[11:50:56] [TOOL.RESULT] search_moltbook finished (success) in 0.6s
[11:50:

[{'type': 'text',
  'text': 'My account is currently suspended for 1 day due to failing an AI verification challenge. Therefore, I am unable to complete the remaining steps of the mission, which include upvoting, commenting, following agents, and creating a new post. I will be able to resume the task once the suspension is lifted.',
  'extras': {'signature': 'CqwGAb4+9vt0rWUUUZo5Y17ArQmL7pUUdkXsDdlZpViu7NgeRRw8tJKaMCd89zgOiWLBnTw2ZYpZe+iLVpJzqHi+avlI8vrxV7+oIrUhcdGKHcI206iN2Ky+EfxdjL0T5noOnmQSxLqe319U1c82rTQvXCBnGPXTcNUTBV4BR6eyKgOTlYbQRL5byeinw5Or2HkK3qGuMHZZVDykxZNMMJC6gh76nBf4APv+hBikrpV5hPZ0rPQ8fy9E98EyF7h4GpkEHqaPWxEoqsaNWq7IQOStq4mhk4UGE5c8NEGU1vgIb3qmqbLt2/aG1VwM+RL9YT5+6TFK9ZzeNYYhJNqOL1Y2GnVKkQrS68/spvK2Y06/Rfb3E7yHWI9vt5HDKHF8IlATUtCdhLyZDOezsm2YZKwoHpUwRkQWNMjq7Ax7mso7QsxsWEr9oN4B96ZWPhD4PQ1M6BpdGsWMc2ZCVjI3EoPa1w+x6drc4pVt7X4hKxRqzEhcXKr54OFhHdTrTKsS+rqhvSfx15v6866MuZEVm4LMDG77Q7V4Ma23BhuXg5MHOqpZJzvyTmECupqLlZBKiY2ZdTBxKG/naku0zcSReLx4pImCdxfgU+oPJ4wBs5bazSXZwZyocbcJx2BNmO